In [ ]:
import sys
sys.path.append("..")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy import signal
import scipy.io.wavfile as wavfile
from scipy.signal import resample, correlate, hilbert
from PIL import Image


from util.plotting import compute_fft_plot_from_sample_rate
from util.data_io import read_rtl_raw_data, read_gqrx_raw_data
from util.filtering import low_pass_filter_complex_signal, low_pass_filter_real_signal
from util.demodulation import chunked_demodulate_signal
from util.phase_locked_loop import phase_locked_loop

In [ ]:
class LowPassFilter():
    def __init__(self, frequency_cutoff: float, sample_rate: int, order: int) -> None:
        self.b,self.a = signal.butter(N=order, Wn=frequency_cutoff, fs=sample_rate)
        self.y = np.zeros((len(self.a) - 1,))
        self.x = np.zeros((len(self.b),))
    
    def step(self, x: float):
        self.x = np.concatenate([[x], self.x[:-1]])
        y = (1/self.a[0]) * (self.x.dot(self.b) - self.y.dot(self.a[1:]))
        self.y = np.concatenate([[y], self.y[:-1]])

        return y

In [ ]:
sample_rate = 1000
length = 60 * sample_rate
time = np.arange(length) / sample_rate
test_signal = np.zeros((length,), dtype=float)

frequencies = np.linspace(25, 250, num=10)
for f in frequencies:
    test_signal += np.cos(2 * np.pi * f * time)

In [ ]:
f,m = compute_fft_plot_from_sample_rate(test_signal, sampling_rate=sample_rate)

fig = go.Figure()
fig.add_scattergl(x=f,y=m)
fig.show()

In [ ]:
lpf = LowPassFilter(frequency_cutoff=150, sample_rate=sample_rate, order=10)


In [ ]:
f,m = signal.freqz(b=lpf.b, a=lpf.a, fs=sample_rate)

fig = go.Figure()
fig.add_scattergl(x=f,y=np.abs(m))
fig.show()

In [ ]:
lpf.b

In [ ]:
lpf.a

In [ ]:
np.set_printoptions(suppress=True, precision=3)

filtered = []

i = 0
for x in test_signal:
    filtered.append(lpf.step(x))
    # i+=1
    # if i > 5:
    #     break

filtered = np.array(filtered)

In [ ]:
f,m = compute_fft_plot_from_sample_rate(filtered, sampling_rate=sample_rate)

fig = go.Figure()
fig.add_scattergl(x=f,y=m)
fig.show()

In [ ]:
filtered_alt = signal.lfilter(b=lpf.b, a=lpf.a, x=test_signal)


f,m = compute_fft_plot_from_sample_rate(filtered_alt, sampling_rate=sample_rate)

fig = go.Figure()
fig.add_scattergl(x=f,y=m)
fig.show()